# Machine Learning: Text Classification Assignment

In [0]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split as tts
from nltk.corpus.reader.plaintext import CategorizedPlaintextCorpusReader

In [0]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Use the CategorizedPlaintextCorpusReader to import the AP_News corpus.

In [0]:
DOC_PATTERN = r'.*\.txt'
CAT_PATTERN = r'([\w_\s]+)/.*'

PATH = '/content/drive/My Drive/DSI Month 4/Week 13/NLP text sets/AP_News'

corpus = CategorizedPlaintextCorpusReader(PATH, DOC_PATTERN, cat_pattern=CAT_PATTERN)

### Create two separate lists - one containing the text from each document and another containing the category of each article in the corpus.

In [0]:
docs = [corpus.raw(fileid) for fileid in corpus.fileids()]
categories = [corpus.categories(fileid)[0] for fileid in corpus.fileids()]

In [0]:
docs[:3]

['HONOLULU (AP) — The University of Hawaii seeks additional funds for student mental health services, scholarships and other items in a new supplementary budget request, officials said. The Board of Regents approved the fiscal year 2020-2021 supplemental operating budget of about $28 million Thursday, The Honolulu Star-Advertiser reported. The request will be submitted to the state Legislature and Democratic Gov. The university requested $2.6 million to hire 19 psychologists for the 10-campus system. The University of Hawaii Manoa has eight psychologists, the Hilo campus has three, and the West Oahu campus has 1.75 positions, while each community college has one position, said Allyson Tanouye, who coordinates mental health throughout the university system. "The national standard is one mental health professional per 1,000 to 1,500 students," Tanouye said. "If we add the 19 positions we will be up to one per 1,500. That\'s how low we are". The mental health funding would also expand pro

In [0]:
categories[:3]

['health', 'health', 'health']

### Preprocess the corpus, ensuring to include the following steps.

- Word tokenize the documents.
- Lemmatize, stem, and lowercase all tokens.
- Remove punctuation and stop words.

In [0]:
def preprocess(docs):
    lemmatizer = WordNetLemmatizer()
    stemmer = SnowballStemmer('english')
    preprocessed = []
    
    for doc in docs:
        tokenized = word_tokenize(doc)

        cleaned = [stemmer.stem(lemmatizer.lemmatize(token.lower())) 
                   for token in tokenized 
                   if not token.lower() in stopwords.words('english') 
                   if token.isalpha()]

        untokenized = " ".join(cleaned)
        preprocessed.append(untokenized)
        
    return preprocessed

In [0]:
preprocessed = preprocess(docs)

In [0]:
preprocessed[0]

'honolulu ap univers hawaii seek addit fund student mental health servic scholarship item new supplementari budget request offici said board regent approv fiscal year supplement oper budget million thursday honolulu report request submit state legislatur democrat gov univers request million hire psychologist system univers hawaii manoa eight psychologist hilo campus three west oahu campus posit communiti colleg one posit said allyson tanouy coordin mental health throughout univers system nation standard one mental health profession per student tanouy said add posit one per low mental health fund would also expand program prevent suicid reduc mental health stigma provid peer educ alert new student parent colleg transit challeng offici said largest item supplement budget million expand hawaii promis program scholarship state institut univers propos flat amount cover tuition fee hawaii resid qualifi feder pell grant look focus needi student go campus said donald straney vice presid academ

### Split the data into training and testing sets with the size of the test set being 30% of the records.

In [0]:
x_train, x_test, y_train, y_test = tts(preprocessed, categories, test_size=0.3)

### Construct a pipeline that TF-IDF vectorizes the text and trains a Random Forest classification model.

In [0]:
# model = Pipeline([
#     ('tfidf', TfidfVectorizer()),
#     ('clf', RandomForestClassifier(n_estimators=100)),
# ])

model = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier(n_estimators=100)),
])

model.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                

### Generate predictions on the test set and print a classification report to evaluate how well the model performed.

In [0]:
predictions = model.predict(x_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

      health       0.79      0.83      0.81        18
    politics       0.73      0.85      0.79        13
      sports       1.00      1.00      1.00        17
        tech       0.93      0.78      0.85        18

    accuracy                           0.86        66
   macro avg       0.86      0.86      0.86        66
weighted avg       0.87      0.86      0.86        66



### Perform 10-fold cross validation and obtain the averge F1 score across all the folds.

In [0]:
scores = cross_val_score(model, preprocessed, categories, 
                         cv=10, scoring='f1_macro')

print(scores.mean())
print(scores.std())

0.7677396214896215
0.09002211247645452


### Ingest, preprocess, and predict the topic of the article at the following URL.

In [0]:
url = 'https://www.nytimes.com/2019/11/25/business/uber-london.html'

In [0]:
import requests
from bs4 import BeautifulSoup

def get_url_text(url):
    response = requests.get(url)
    content = response.text
    
    TAGS = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'p', 'li']
    soup = BeautifulSoup(content, "lxml")
    text_list = [tag.get_text() for tag in soup.find_all(TAGS)]
    text = ' '.join(text_list)
    return text

In [0]:
text = get_url_text(url)

In [0]:
cleaned = preprocess([text])

In [0]:
cleaned

['advertis support uber fight surviv london lose licens compani odd regul driver tradit cab lucrat european market adam satariano ami tsang london uber suffer major blow monday london transport author made surpris decis extend taxi oper licens persist safeti problem throw question whether compani continu oper lucrat european market decis immedi affect uber presenc london street compani said would appeal decis set could long legal process uber continu oper throughout time news add difficult year compani stage disappoint initi public offer may uber sinc experienc execut turnov sever round layoff face continu public scrutini safeti passeng transport london regul taxi privat hire servic citi said uber meet fit proper standard need hold taxi licens regul said uber pattern failur place passeng safeti risk includ vulner uber app allow unauthor driver carri thousand rider clear concern issu aros also concern confid similar issu happen helen chapman director licens regul charg transport london 

In [0]:
model.predict(cleaned)[0]

'tech'